In [2]:
import requests
import pandas as pd
from datetime import datetime, timedelta

# Replace with your Alpha Vantage API key
api_key = 'YOUR_API_KEY'

# Function to fetch forex data
def fetch_forex_data(from_currency, to_currency):
    url = f'https://www.alphavantage.co/query'
    params = {
        'function': 'FX_DAILY',
        'from_symbol': from_currency,
        'to_symbol': to_currency,
        'apikey': api_key,
        'outputsize': 'full'
    }
    response = requests.get(url, params=params)
    data = response.json()
    
    # Extracting the time series data
    time_series = data.get('Time Series FX (Daily)', {})
    
    # Converting to DataFrame
    df = pd.DataFrame.from_dict(time_series, orient='index')
    df.index = pd.to_datetime(df.index)
    df = df.sort_index()
    
    # Adding a column for the currency pair
    df['Currency Pair'] = f'{from_currency}/{to_currency}'
    
    # Filtering data for the last 10 years
    ten_years_ago = datetime.now() - timedelta(days=365*10)
    df = df[df.index >= ten_years_ago]
    
    return df

# List of common currency pairs to fetch
currency_pairs = [
    ('USD', 'INR'), ('EUR', 'USD'), ('GBP', 'USD'), ('USD', 'JPY'), ('AUD', 'USD'),
    ('USD', 'CAD'), ('USD', 'CHF'), ('NZD', 'USD'), ('EUR', 'GBP'), ('EUR', 'JPY'),
    ('GBP', 'JPY'), ('AUD', 'JPY'), ('EUR', 'AUD'), ('GBP', 'AUD'), ('USD', 'SGD')
]

# Fetching data for all currency pairs and combining into a single DataFrame
all_data = pd.DataFrame()
for from_currency, to_currency in currency_pairs:
    forex_data = fetch_forex_data(from_currency, to_currency)
    all_data = pd.concat([all_data, forex_data])

# Resetting index for the combined DataFrame
all_data.reset_index(inplace=True)
all_data.rename(columns={'index': 'Date'}, inplace=True)

print(all_data)


            Date   1. open   2. high    3. low  4. close Currency Pair
0     2014-09-22  60.93800  61.04800  60.59000  61.00000       USD/INR
1     2014-09-23  61.04800  61.04800  60.84000  60.98000       USD/INR
2     2014-09-24  60.81000  61.46000  60.81000  60.81000       USD/INR
3     2014-09-25  61.49000  61.60000  61.01300  61.49000       USD/INR
4     2014-09-28  61.38000  61.62000  61.13800  61.13800       USD/INR
...          ...       ...       ...       ...       ...           ...
39074 2024-09-11   1.30456   1.30589   1.30320   1.30456       USD/SGD
39075 2024-09-12   1.30068   1.30067   1.29740   1.30068       USD/SGD
39076 2024-09-15   1.29743   1.29791   1.29400   1.29743       USD/SGD
39077 2024-09-16   1.29566   1.29660   1.29240   1.29566       USD/SGD
39078 2024-09-18   1.29590   1.29590   1.29357   1.29427       USD/SGD

[39079 rows x 6 columns]
